In [1]:
# problem 파일 경로 Desktop/project/code_similarity/open/code

# problem 폴더 안 예제 코드 확장자명 변경
import os
import glob

# .txt 병합
import pandas as pd
import numpy as np
import sklearn

# from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import scipy

In [2]:
# 폴더 및 .py 파일 개수 확인
path = "./open/code/"
dataset = os.listdir(path)

In [3]:
# ds.store숨김
dataset = [folder for folder in dataset if folder.startswith('problem')]

In [4]:
# 폴더명 정렬
dataset.sort(reverse=False)
len(dataset)

300

In [ ]:
# code - code/problem/.py 모두 불러옴
'''
cnt = 0
for i in range(len(dataset)):
    print(f"{dataset[i]} \t : \t {len(os.listdir(path + dataset[i]))}")
    cnt += len(os.listdir(path + dataset[i]))
'''

In [ ]:
# .py 읽어오기
# file = glob.glob(path + "*/*.py")

In [ ]:
# 파일명 .txt 변환
'''for name_ext in file:
    if not os.path.isdir(name_ext):
        src = os.path.splitext(name_ext)
        os.rename(name_ext, src[0]+'.txt')
'''

In [5]:
# txt 파일 DF 저장
csv_tr = []
csv_tr2 = []
similarity = []
for i in glob.glob("./open/code/*/*.txt"):
    with open(os.path.join(os.getcwd(), i), 'r') as f:
        text = f.read()
        #무작위 행 셔플
        csv_tr.append(text)
        csv_tr2.append(text)
        similarity.append(-1)
csv_tr = sklearn.utils.shuffle(csv_tr, random_state = 42)

In [6]:
csv_tr[0]

"X = int(input())\n\nif X >= 400 and X < 600:\n    print('8')\nelif X >= 600 and X < 800:\n    print('7')\nelif X >= 800 and X < 1000:\n    print('6')\nelif X >= 1000 and X < 1200:\n    print('5')\nelif X >= 1200 and X < 1400:\n    print('4')\nelif X >= 1400 and X < 1600:\n    print('3')\nelif X >= 1600 and X < 1800:\n    print('2')\nelif X >= 1800 and X < 2000:\n    print('1')"

In [7]:
# DF 생성
code_csv = pd.DataFrame(zip(csv_tr, csv_tr2,similarity),
                        columns=['code1', 'code2','similarity'])
#code_csv.to_csv("code_csv.csv")

In [ ]:
'''
# Load Train / Test dataset
# train, test 선언
train = code_csv
train.head()
train.info()
'''

In [ ]:
'''test = pd.read_csv("./open/test.csv")
test.info()
test.head()
'''

In [8]:
# %% define Model

# TF-idf행렬의 크기
tfidV = TfidfVectorizer()

In [9]:
tfidV_matrix1 = tfidV.fit(code_csv['code1'])  # fit(train['code1'])
tfidV_matrix2 = tfidV.fit(code_csv['code2'])


In [10]:
test = pd.read_csv("./open/test.csv")

In [11]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179700 entries, 0 to 179699
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   pair_id  179700 non-null  int64 
 1   code1    179700 non-null  object
 2   code2    179700 non-null  object
dtypes: int64(1), object(2)
memory usage: 4.1+ MB


In [12]:
test.head()

,pair_id,code1,code2
0,1,def main():\n s = input()\n if s.count('a') ...,"N,K = map(int,input().split())\nA = list(map(i..."
1,2,"N,K,Q = map(int,input().split())\npoints = [0]...","N, K, Q = map(int,input().split())\n\nif K > Q..."
2,3,from itertools import combinations\nn = int(in...,s = input()\nt = input()\nlength_s = len(s)\nl...
3,4,"a,b=map(int,input().split())\n\nans1=a+b\nans2...","a, b, c, d = map(int,input().split())\n\nif a ..."
4,5,S = input()\nK = int(input())\n\nind = -1\nfor...,"H, W = map(int, input().split())\ngrid = []\nf..."


In [13]:
tfidV_test1 = tfidV.transform(test['code1'])  # fit(train['code1'])
tfidV_test2 = tfidV.transform(test['code2'])

print("행렬의 크기: ", tfidV_test1.shape)
print("행렬의 크기: ", tfidV_test2.shape)

행렬의 크기:  (179700, 44705)
행렬의 크기:  (179700, 44705)


In [14]:
preds = []
for code1_vec, code2_vec in zip(tfidV_test1, tfidV_test2):
    preds.append(cosine_similarity(code1_vec, code2_vec))

In [15]:
preds = np.reshape(preds, len(preds))
len(preds)

179700

In [36]:
submission = pd.read_csv('./open/sample_submission.csv')
#submission = submission.iloc[:45588]
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179700 entries, 0 to 179699
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   pair_id  179700 non-null  int64
 1   similar  179700 non-null  int64
dtypes: int64(2)
memory usage: 2.7 MB


In [37]:
submission['similar'] = preds

In [44]:
threshold = 0.5
submission['similar'] = np.where(submission['similar'] > threshold, 1, 0)

In [49]:
submission['similar'].value_counts()

0    164250
1     15450
Name: similar, dtype: int64

In [51]:
submission.to_csv('./svalue_countsion.csv', index=False)

In [ ]:
# Define Model (CountVectorizer+CosineSimilarity)
class BaselineModel():
    def __init__(self, threshold=0.5):
        super(BaselineModel, self).__init__()
        self.threshold = threshold  # 유사도 임계값
        self.TfidfVectorizer = TfidfVectorizer()

    def fit(self, code1, code2):
        # 입력 받은 코드 쌍으로 부터 vectorizer를 vector화
        code1 = self.TfidfVectorizer.fit_transform(code1)
        code2 = self.TfidfVectorizer.fit_transform(code2)
        print('Done1.')

    def predict_proba(self, code1, code2):

        # 입력 받은 코드 쌍으로 부터 vectorizer를 vector화
        code1_vecs = self.TfidfVectorizer.fit_transform(code1)
        code2_vecs = self.TfidfVectorizer.fit_transform(code2)

        preds = []

        # 각각의 코드 쌍(=벡터 쌍)으로부터 cosine-similarity를 구합니다.
        for code1_vec, code2_vec in zip(code1_vecs, code2_vecs):
            preds.append(cosine_similarity(code1_vec, code2_vec))

        # preds = np.reshape(preds, len(preds))
        preds = np.reshape(preds, len(preds))

        print('Done2.')

        # 각 코드 쌍들의 유사도를 반환
        return preds

    def predict(self, code1, code2):
        preds = self.predict_proba(code1, code2)

        # cosine-similarity (유사도)가 설정한 임계값(Threshold=0.5)보다 높다면 유사하다 : 1, 아니라면 유사하지 않다 : 0
        preds = np.where(preds > self.threshold, 1, 0)

        # 각 코드 쌍들의 유사도를 Threshold를 통해 유사함을 판별 (이진분류)
        return preds

In [ ]:
# Model(Vectorizer) Fit


# 모델 선언
model = BaselineModel(threshold=0.5)

# 학습 코드 쌍들로부터 Model을 Fitting
model_csv = model.fit(train['code1'], train['code2'])


# Inference
# 모델 추론
preds = model.predict(test['code1'], test['code2'])
# Submission

In [ ]:
submission = pd.read_csv('./open/sample_submission.csv')
submission['similar'] = preds
submission.to_csv('./submission.csv', index=False)